In [25]:
import pandas as pd
conditions_df = pd.read_csv("covid-data/conditions.csv")
patients_df = pd.read_csv("covid-data/patients.csv")
observations_df = pd.read_csv("covid-data/observations.csv")
care_plans_df = pd.read_csv("covid-data/careplans.csv")
encounters_df = pd.read_csv("covid-data/encounters.csv")
procedures_df = pd.read_csv("covid-data/procedures.csv")
medications_df = pd.read_csv("covid-data/medications.csv")

In [26]:
immunizations_df = pd.read_csv("covid-data/immunizations.csv")
allergies_df = pd.read_csv("covid-data/allergies.csv")

In [13]:
patients_df.columns

Index(['Id', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX',
       'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY',
       'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP',
       'LAT', 'LON', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE'],
      dtype='object')

In [16]:
observations_df.columns

Index(['DATE', 'PATIENT', 'ENCOUNTER', 'CODE', 'DESCRIPTION', 'VALUE', 'UNITS',
       'TYPE'],
      dtype='object')

In [18]:
immunizations_df.columns

Index(['DATE', 'PATIENT', 'ENCOUNTER', 'CODE', 'DESCRIPTION', 'BASE_COST'], dtype='object')

In [19]:
allergies_df.columns

Index(['START', 'STOP', 'PATIENT', 'ENCOUNTER', 'CODE', 'DESCRIPTION'], dtype='object')

In [20]:
care_plans_df.columns

Index(['Id', 'START', 'STOP', 'PATIENT', 'ENCOUNTER', 'CODE', 'DESCRIPTION',
       'REASONCODE', 'REASONDESCRIPTION'],
      dtype='object')

In [21]:
medications_df.columns

Index(['START', 'STOP', 'PATIENT', 'PAYER', 'ENCOUNTER', 'CODE', 'DESCRIPTION',
       'BASE_COST', 'PAYER_COVERAGE', 'DISPENSES', 'TOTALCOST', 'REASONCODE',
       'REASONDESCRIPTION'],
      dtype='object')

In [36]:
def generate_patient_summary(patient_id,type='summary'):
    # Extract patient information
    patient_info = patients_df[patients_df['Id'] == patient_id].iloc[0]
    observations = observations_df[observations_df['PATIENT'] == patient_id]
    immunizations = immunizations_df[immunizations_df['PATIENT'] == patient_id]
    careplans = care_plans_df[care_plans_df['PATIENT'] == patient_id]
    medications = medications_df[medications_df['PATIENT'] == patient_id]
    allergies = allergies_df[allergies_df['PATIENT'] == patient_id]

    if type == 'summary':
        # Generate a patient summary
        
        # Construct the prompt
        prompt = f"""
        Patient Summary:
        - Birth date: {patient_info['BIRTHDATE']}
        - Gender: {patient_info['GENDER']}
        - Observations: {', '.join(observations['DESCRIPTION'].tolist())}
        - Immunizations: {', '.join(immunizations['DESCRIPTION'].tolist())}
        - Allergies: {', '.join(allergies['DESCRIPTION'].tolist())}
        - Care Plans: {', '.join(careplans['DESCRIPTION'].tolist())}
        - Medications: {', '.join(medications['DESCRIPTION'].tolist())}

        Based on the above information, provide a comprehensive summary of the patient's health status.
        """
    elif type == 'disease':
        prompt = f"""
        Analyze the following patient data step-by-step to assess disease risk:
        
        - Birth date: {patient_info['BIRTHDATE']}
        - Gender: {patient_info['GENDER']}
        - Observations: {', '.join(observations['DESCRIPTION'].tolist())}
        - Immunizations: {', '.join(immunizations['DESCRIPTION'].tolist())}
        - Allergies: {', '.join(allergies['DESCRIPTION'].tolist())}
        - Care Plans: {', '.join(careplans['DESCRIPTION'].tolist())}
        - Medications: {', '.join(medications['DESCRIPTION'].tolist())}

        Step-by-step analysis:
        """
    #print(prompt)
    return prompt
# Generate a patient summary for the first patient
print(generate_patient_summary("f0f3bc8d-ef38-49ce-a2bd-dfdda982b271"))


        Patient Summary:
        - Birth date: 2017-08-24
        - Gender: M
        - Observations: Body Height, Pain severity - 0-10 verbal numeric rating [Score] - Reported, Body Weight, Weight-for-length Per age and sex, Head Occipital-frontal circumference, Diastolic Blood Pressure, Systolic Blood Pressure, Heart rate, Respiratory rate, Tobacco smoking status NHIS, Body Height, Pain severity - 0-10 verbal numeric rating [Score] - Reported, Body Weight, Weight-for-length Per age and sex, Head Occipital-frontal circumference, Body Mass Index, Body mass index (BMI) [Percentile] Per age and gender, Diastolic Blood Pressure, Systolic Blood Pressure, Heart rate, Respiratory rate, Tobacco smoking status NHIS, Body temperature, Respiratory rate, Heart rate, Oxygen saturation in Arterial blood, Diastolic Blood Pressure, Systolic Blood Pressure, Body Weight, Influenza virus A RNA [Presence] in Respiratory specimen by NAA with probe detection, Influenza virus B RNA [Presence] in Respirator

In [28]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [32]:

from openai import OpenAI

client = OpenAI(
  organization='org-GXoJKN5nYBp1bYCBJjxDxF9I',
  project='proj_sCh97WttAzvfhWXyvoOofMLh',
)


def get_summary(prompt):
    # Call the OpenAI API
    completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a doctor specializing in covid-19."},
        {
            "role": "user",
            "content": prompt
        }
    ]
    )   
    #print(completion.choices[0].message)
    return completion.choices[0].message.content
summary = get_summary(generate_patient_summary("f0f3bc8d-ef38-49ce-a2bd-dfdda982b271"))
print(summary)


ChatCompletionMessage(content="**Patient Health Status Summary**\n\n**Demographics:**\n- **Birth Date:** August 24, 2017\n- **Gender:** Male\n\n**Clinical Observations:**\n- The patient has undergone multiple clinical evaluations and screenings, including vital signs and respiratory assessments, which comprise:\n  - **Body Height**: Measurement recorded.\n  - **Body Weight**: Measurement recorded.\n  - **Weight-for-Length**: Evaluated per age and sex.\n  - **Head Circumference**: Recorded measurements of occipital-frontal circumference.\n  - **Blood Pressure**: Both systolic and diastolic blood pressure were measured.\n  - **Heart Rate and Respiratory Rate**: Both parameters recorded.\n  - **Oxygen Saturation**: Assessed in arterial blood.\n  - **Body Temperature**: Recorded.\n\n**Pain Assessment:**\n- Reported pain severity on a 0-10 verbal numeric rating scale, indicating the patient's experience of pain.\n\n**Infectious Disease Screening:**\n- Testing for the presence of various vir

In [35]:
# Implement Chain of though reasoning to articulate its reasoining step-by-step
disease_analyis = get_summary(generate_patient_summary("f0f3bc8d-ef38-49ce-a2bd-dfdda982b271",type='disease'))
print(disease_analyis)

ChatCompletionMessage(content="To assess the COVID-19 disease risk for the patient described based on the data provided, I'll analyze the relevant factors step-by-step:\n\n### Step 1: Patient Demographics\n- **Birth date**: 2017-08-24\n- **Gender**: Male\n\n**Analysis**: The child is currently 6 years old. Pediatric patients, especially those under 12, may have different risks and manifestations of COVID-19 compared to adults. Generally, children experience milder disease and lower hospitalization rates; however, they can still be infectious and transmit the virus.\n\n### Step 2: Immunization Status\n- **Vaccinations**: Influenza seasonal injectable preservative free, Hepatitis A pediatric/adolescent (2 doses)\n\n**Analysis**: The patient appears to be up-to-date with routine vaccinations for influenza and hepatitis A. However, there is no mention of COVID-19 vaccination. The COVID-19 vaccine is recommended for eligible children (children aged 6 months and above). If the child has not 

In [37]:
# Above patient was diagnosed with COVID-19, run the summary and analysis for the patient who died of COVID-19
not_alive = patients_df[patients_df['DEATHDATE'].notnull()][0:1]
print(not_alive)


                                      Id   BIRTHDATE   DEATHDATE          SSN  \
11  58cac9ec-4baa-46c1-b919-0ed13572b51d  1970-06-13  2016-05-07  999-90-2484   

      DRIVERS    PASSPORT PREFIX      FIRST         LAST SUFFIX  ...  \
11  S99989337  X48630763X    Ms.  Isabel214  Carvajal694    NaN  ...   

                         BIRTHPLACE                     ADDRESS        CITY  \
11  Santiago  Santiago Province  CL  335 Morar Highlands Apt 75  Fall River   

            STATE          COUNTY     ZIP        LAT        LON  \
11  Massachusetts  Bristol County  2720.0  41.687684 -71.055769   

   HEALTHCARE_EXPENSES HEALTHCARE_COVERAGE  
11          1145378.45              4941.2  

[1 rows x 25 columns]


In [38]:
#58cac9ec-4baa-46c1-b919-0ed13572b51d
print(generate_patient_summary("58cac9ec-4baa-46c1-b919-0ed13572b51d"))


        Patient Summary:
        - Birth date: 1970-06-13
        - Gender: F
        - Observations: Cause of Death [US Standard Certificate of Death]
        - Immunizations: 
        - Allergies: 
        - Care Plans: Diabetes self management plan
        - Medications: Clopidogrel 75 MG Oral Tablet, Alteplase 100 MG Injection

        Based on the above information, provide a comprehensive summary of the patient's health status.
        


In [39]:
print(get_summary(generate_patient_summary("58cac9ec-4baa-46c1-b919-0ed13572b51d")))

ChatCompletionMessage(content='Based on the provided information, here is a comprehensive summary of the patient\'s health status:\n\n**Patient Profile:**\n- **Age:** As of October 2023, the patient is 53 years old.\n- **Gender:** Female\n\n**Medical History:**\n- The patient has a care plan in place for diabetes self-management, indicating that she has a diagnosis of diabetes. This can affect her overall health, management of other conditions, and response to treatments.\n- There is no information on the specific type of diabetes (Type 1 or Type 2) or details on her blood glucose control, which would be important in understanding her health status.\n\n**Medications:**\n1. **Clopidogrel 75 MG Oral Tablet:** Clopidogrel is an antiplatelet medication used to prevent blood clots. Its use suggests that the patient may have a history of cardiovascular issues, such as heart disease, a previous heart attack, or stroke.\n2. **Alteplase 100 MG Injection:** Alteplase is a thrombolytic agent used

In [40]:
# disease analysis
print(get_summary(generate_patient_summary("58cac9ec-4baa-46c1-b919-0ed13572b51d",type='disease')))

ChatCompletionMessage(content="To analyze this patient data step-by-step regarding the risk of severe outcomes from COVID-19, we will consider each factor listed in the patient's profile.\n\n### Step 1: Basic Demographics\n- **Birth date**: June 13, 1970\n  - **Age**: As of today (October 2023), the patient is 53 years old. \n  - **Age Factor**: Individuals aged 50 and older are at increased risk for severe illness from COVID-19.\n\n- **Gender**: Female\n  - **Gender Consideration**: Studies have shown that males may have a higher risk of severe COVID-19 outcomes compared to females, although females still can experience significant risks depending on other conditions.\n\n### Step 2: Health Conditions\n- **Observations**: Cause of Death [US Standard Certificate of Death]\n  - Without specific details regarding the cause of death, we cannot determine the implications for current risk.\n  \n- **Care Plans**: Diabetes self-management plan\n  - **Diabetes**: Diabetes is a recognized condit

In [41]:
# find out if any procedure was performed on the patient
procedures_df[procedures_df['PATIENT'] == "58cac9ec-4baa-46c1-b919-0ed13572b51d"]

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION
86,2016-05-07,58cac9ec-4baa-46c1-b919-0ed13572b51d,15bcb310-ccbd-4371-bfe6-e9b79ec74aa9,40701008,Echocardiography (procedure),819.51,230690007.0,Stroke
87,2016-05-07,58cac9ec-4baa-46c1-b919-0ed13572b51d,15bcb310-ccbd-4371-bfe6-e9b79ec74aa9,433112001,Percutaneous mechanical thrombectomy of portal...,27197.09,230690007.0,Stroke


In [42]:
# add procedure to the patient summary to find out the impact of the procedure on the patient
def generate_patient_procedure_summary(patient_id,type='summary'):
    # Extract patient information
    patient_info = patients_df[patients_df['Id'] == patient_id].iloc[0]
    observations = observations_df[observations_df['PATIENT'] == patient_id]
    immunizations = immunizations_df[immunizations_df['PATIENT'] == patient_id]
    careplans = care_plans_df[care_plans_df['PATIENT'] == patient_id]
    medications = medications_df[medications_df['PATIENT'] == patient_id]
    allergies = allergies_df[allergies_df['PATIENT'] == patient_id]
    procedures = procedures_df[procedures_df['PATIENT'] == patient_id]

    if type == 'summary':
        # Generate a patient summary
        
        # Construct the prompt
        prompt = f"""
        Patient Summary:
        - Birth date: {patient_info['BIRTHDATE']}
        - Gender: {patient_info['GENDER']}
        - Observations: {', '.join(observations['DESCRIPTION'].tolist())}
        - Immunizations: {', '.join(immunizations['DESCRIPTION'].tolist())}
        - Allergies: {', '.join(allergies['DESCRIPTION'].tolist())}
        - Care Plans: {', '.join(careplans['DESCRIPTION'].tolist())}
        - Medications: {', '.join(medications['DESCRIPTION'].tolist())}
        - Procedures: {', '.join(procedures['DESCRIPTION'].tolist())}

        Based on the above information, provide a comprehensive summary of the patient's health status.
        """
    elif type == 'disease':
        prompt = f"""
        Analyze the following patient data step-by-step to assess disease risk:
        
        - Birth date: {patient_info['BIRTHDATE']}
        - Gender: {patient_info['GENDER']}
        - Observations: {', '.join(observations['DESCRIPTION'].tolist())}
        - Immunizations: {', '.join(immunizations['DESCRIPTION'].tolist())}
        - Allergies: {', '.join(allergies['DESCRIPTION'].tolist())}
        - Care Plans: {', '.join(careplans['DESCRIPTION'].tolist())}
        - Medications: {', '.join(medications['DESCRIPTION'].tolist())}
        - Procedures: {', '.join(procedures['DESCRIPTION'].tolist())}

        Step-by-step analysis:
        """
    #print(prompt)
    return prompt
# Generate a patient summary for the first patient
print(generate_patient_procedure_summary("f0f3bc8d-ef38-49ce-a2bd-dfdda982b271"))


        Patient Summary:
        - Birth date: 2017-08-24
        - Gender: M
        - Observations: Body Height, Pain severity - 0-10 verbal numeric rating [Score] - Reported, Body Weight, Weight-for-length Per age and sex, Head Occipital-frontal circumference, Diastolic Blood Pressure, Systolic Blood Pressure, Heart rate, Respiratory rate, Tobacco smoking status NHIS, Body Height, Pain severity - 0-10 verbal numeric rating [Score] - Reported, Body Weight, Weight-for-length Per age and sex, Head Occipital-frontal circumference, Body Mass Index, Body mass index (BMI) [Percentile] Per age and gender, Diastolic Blood Pressure, Systolic Blood Pressure, Heart rate, Respiratory rate, Tobacco smoking status NHIS, Body temperature, Respiratory rate, Heart rate, Oxygen saturation in Arterial blood, Diastolic Blood Pressure, Systolic Blood Pressure, Body Weight, Influenza virus A RNA [Presence] in Respiratory specimen by NAA with probe detection, Influenza virus B RNA [Presence] in Respirator

In [43]:
# Generate a patient summary for the patient died
print(generate_patient_procedure_summary("58cac9ec-4baa-46c1-b919-0ed13572b51d"))


        Patient Summary:
        - Birth date: 1970-06-13
        - Gender: F
        - Observations: Cause of Death [US Standard Certificate of Death]
        - Immunizations: 
        - Allergies: 
        - Care Plans: Diabetes self management plan
        - Medications: Clopidogrel 75 MG Oral Tablet, Alteplase 100 MG Injection
        - Procedures: Echocardiography (procedure), Percutaneous mechanical thrombectomy of portal vein using fluoroscopic guidance

        Based on the above information, provide a comprehensive summary of the patient's health status.
        


In [44]:
# step-by-step analysis
print(get_summary(generate_patient_procedure_summary("58cac9ec-4baa-46c1-b919-0ed13572b51d",type='disease')))

ChatCompletionMessage(content="To assess the disease risk of this patient, we will analyze her demographic information, medical history, current medications, and any ongoing care plans. \n\n### Step 1: Demographic Information\n- **Birth Date**: 1970-06-13\n    - Age: As of October 2023, the patient is 53 years old. COVID-19 severity tends to increase with age, particularly in patients over 50. This age group, therefore, constitutes a moderate to high-risk group for severe outcomes associated with COVID-19.\n    \n- **Gender**: Female\n    - While both genders can be severely affected by COVID-19, some studies have shown that men may have a slightly higher risk of severe illness. However, underlying health conditions play a more critical role. \n\n### Step 2: Existing Medical Conditions\n- **Care Plans**: Diabetes self-management plan\n    - Diabetes is a known risk factor for severe illness from COVID-19. Uncontrolled diabetes can further elevate this risk, making effective management 

In [46]:
def get_patient_info(patient_id):
    patient_info = patients_df[patients_df['Id'] == patient_id].iloc[0]
    observations = observations_df[observations_df['PATIENT'] == patient_id]
    immunizations = immunizations_df[immunizations_df['PATIENT'] == patient_id]
    careplans = care_plans_df[care_plans_df['PATIENT'] == patient_id]
    medications = medications_df[medications_df['PATIENT'] == patient_id]
    allergies = allergies_df[allergies_df['PATIENT'] == patient_id]
    procedures = procedures_df[procedures_df['PATIENT'] == patient_id]
    return f"""
    - Birth date: {patient_info['BIRTHDATE']}
        - Gender: {patient_info['GENDER']}
        - Observations: {', '.join(observations['DESCRIPTION'].tolist())}
        - Immunizations: {', '.join(immunizations['DESCRIPTION'].tolist())}
        - Allergies: {', '.join(allergies['DESCRIPTION'].tolist())}
        - Care Plans: {', '.join(careplans['DESCRIPTION'].tolist())}
        - Medications: {', '.join(medications['DESCRIPTION'].tolist())}
        - Procedures: {', '.join(procedures['DESCRIPTION'].tolist())}
"""
print(get_patient_info("58cac9ec-4baa-46c1-b919-0ed13572b51d"))


    - Birth date: 1970-06-13
        - Gender: F
        - Observations: Cause of Death [US Standard Certificate of Death]
        - Immunizations: 
        - Allergies: 
        - Care Plans: Diabetes self management plan
        - Medications: Clopidogrel 75 MG Oral Tablet, Alteplase 100 MG Injection
        - Procedures: Echocardiography (procedure), Percutaneous mechanical thrombectomy of portal vein using fluoroscopic guidance



In [47]:
# based on the patient information, predict the most likely diseases
def predict_disease(patient_summary):
    prompt = (
        "Given the following patient summary, predict the most likely diseases:\n\n"
        f"{patient_summary}\n\n"
        "Predicted diseases:"
    )
    return get_summary(prompt)
print(predict_disease(get_patient_info("58cac9ec-4baa-46c1-b919-0ed13572b51d")))

ChatCompletionMessage(content="Based on the provided patient summary, the following diseases or conditions can be predicted or considered likely:\n\n1. **Diabetes Mellitus**: The care plan indicates a diabetes self-management plan, suggesting that the patient has diabetes, which could be type 1 or type 2.\n\n2. **Cardiovascular Disease**: The use of Clopidogrel (an antiplatelet medication often prescribed to prevent blood clots in patients with cardiovascular diseases) and the echocardiography procedure may suggest a history of cardiovascular issues, such as coronary artery disease or a history of myocardial infarction.\n\n3. **Stroke or Transient Ischemic Attack (TIA)**: The administration of Alteplase (a thrombolytic agent) indicates that the patient may have experienced a stroke or TIA, as this medication is commonly used to dissolve clots in patients suffering from these conditions.\n\n4. **Portal Vein Thrombosis**: The mention of a percutaneous mechanical thrombectomy of the porta

In [48]:
# Generate multiple disease hypotheses
def disease_hypotheses(patient_summary):
    prompt = (
        "Given the following patient summary, generate multiple hypotheses for potential diseases:\n\n"
        f"{patient_summary}\n\n"
        "Disease hypotheses:"
    )
    return get_summary(prompt)
print(disease_hypotheses(get_patient_info("58cac9ec-4baa-46c1-b919-0ed13572b51d")))

ChatCompletionMessage(content="Based on the patient summary provided, several hypotheses for potential diseases or conditions can be generated, considering the patient's age, gender, medications, procedures performed, and existing care plans. Here are possible disease hypotheses:\n\n1. **Cardiovascular Disease**:\n   - The use of Clopidogrel and the echocardiography suggests the patient may have a history of coronary artery disease or acute coronary syndrome.\n   - The percutaneous mechanical thrombectomy indicates the presence of thrombosis, possibly related to cardiovascular conditions.\n\n2. **Diabetes Complications**:\n   - The diabetes self-management plan implies the presence of diabetes mellitus, which could lead to various complications such as cardiovascular disease, neuropathy, or retinopathy.\n   - The presence of vascular complications leading to thrombosis could be related to poorly managed diabetes.\n\n3. **Cerebrovascular Accident (Stroke)**:\n   - The administration of 